In [ ]:
! pip install -q mlflow torchinfo

In [ ]:
import mlflow
import mlflow.pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torch.optim import lr_scheduler
import torchvision
import torchvision.transforms as transforms
from torchinfo import summary
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
# CNN Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 5, padding=2)  # Padding added to keep size as 28x28
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 5, padding=2)  # Padding added to keep size as 14x14
        self.bn2 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)  # Adjusted input size
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = torch.flatten(x, 1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

In [ ]:
train_info = {
    'exp_name': 'Mnist-test',
    'batchSize': 128,
    'img_shape': [3,224,224],
    'epoch': 20,
    'early_stop': 3,
    'lr': 0.00001,
    'optim': 'Adam',
    'secheduler': 'CosineAnnealingLR',
    'loss': 'CrossEntropyLoss'
}

In [ ]:
#Load Dataset(CIFAR10)

transform = transforms.Compose([transforms.ToTensor()])

train = torchvision.datasets.MNIST(root='./', train=True, download=True, transform=transform) 
test = torchvision.datasets.MNIST(root='./', train=False, download=False, transform=transform)

split=0.8
train_size=int(len(train)*split)
traindata, valdata = data.random_split(train, [train_size, len(train)-train_size])
#traindata, valdata = train_test_split(train, test_size = 0.2, random_state = 111)

trainLoader = data.DataLoader(traindata, batch_size=train_info['batchSize'], shuffle=True, num_workers=2)
validLoader = data.DataLoader(valdata, batch_size=train_info['batchSize'], shuffle=True,num_workers=2)
testLoader = data.DataLoader(test, batch_size=train_info['batchSize'], shuffle=False, num_workers=2)

In [ ]:
#visualize img
img = train[1][0]

plt.imshow(img.squeeze(0),cmap='binary')
plt.show()
print(train[1][1])
print(img.size())

In [ ]:
# Parameters

device ='cuda:0' if torch.cuda.is_available() else 'cpu' #check cuda
print(device,'is available!\n')
model = Net().to(device)
#summary(model, input_size=(128, 1, 32, 32))
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

optimizer = optim.Adam(model.parameters(),betas=(0.9, 0.98), lr=train_info['lr'])
scheduler=lr_scheduler.CosineAnnealingLR(optimizer,5)
criterion= nn.CrossEntropyLoss()

In [ ]:
print(summary(model, input_size=(128, 1, 28, 28)))
with open("model_info.txt", "w") as f:
    f.write(summary(model, input_size=(128, 1, 28, 28)).__repr__())

In [ ]:
# Training
#store loss inf
loss={'train_loss':[],'valid_loss':[],'lr':[]}        

#early_stopping
patience=5
earlystop=0
best_loss=float('inf')
epochs = 100


# mlflow set url
mlflow.set_tracking_uri("http://mlflow-web:5000/")

if not mlflow.get_experiment_by_name(train_info['exp_name']):
    mlflow.create_experiment(train_info['exp_name'])

mlflow.set_experiment(train_info['exp_name'])


with mlflow.start_run() as run:
    # para
    mlflow.log_param("data set", "Torchvision-Mnsit")
    mlflow.log_param("Batch Size", train_info['batchSize'])
    if device != 'cpu':
        mlflow.log_param("gpu", "enabled")
    mlflow.log_param("epoch",train_info['epoch'])
    mlflow.log_param("optimizer", train_info['optim'])
    mlflow.log_param("init_lr", train_info['lr'])
    mlflow.log_param("scheduler", train_info['secheduler'])
    mlflow.log_param("loss function", train_info['loss'])
    mlflow.log_artifact("model_info.txt")
    
    for epoch in range(train_info['epoch']):
        #model train
        model.train()
        tmp=[]
        with tqdm(trainLoader,leave=False) as tepoch:
            for inputs, target in tepoch:
                #set tqdm description
                tepoch.set_description(f'TrainEpoch{epoch+1:3d}')
                inputs, target=inputs.to(device), target.to(device)
                # Zero the parameter gradients
                optimizer.zero_grad()
                output = model(inputs)
                crit = criterion(output, target)

                tmp.append(crit.cpu().item())

                crit.backward()
                optimizer.step()
                
                tepoch.set_postfix({'loss':'{0:.3f}'.format(np.average(tmp))})
                
                
            
        scheduler.step()
        mlflow.log_metric("train_loss", np.average(tmp), epoch+1)
        loss['train_loss'].append(np.average(tmp))
        #model valid
        model.eval()
        tmp=[]
        with tqdm(validLoader,leave=False) as vepoch:
            for inputs, target in vepoch:
                vepoch.set_description(f'ValidEpoch{epoch+1:3d}')
                inputs,target=inputs.to(device), target.to(device)

                output=model(inputs)
                crit=criterion(output,target)

                tmp.append(crit.cpu().item())
                vepoch.set_postfix({'loss':'{0:.3f}'.format(np.average(tmp))})

        

        loss['valid_loss'].append(np.average(tmp))
        loss['lr'].append(scheduler.get_last_lr())
        mlflow.log_metric("valid_loss", np.average(tmp), epoch+1)
        mlflow.log_metric("learning_rate", scheduler.get_last_lr()[0], epoch+1)
        # earlystopping
        cur_loss=np.average(tmp)
        if cur_loss>best_loss:
            earlystop+=1
            if earlystop>= patience:
                print('Early stopping!')
                mlflow.log_param("Early_Stopping_Happened", f'finish in {epoch+1} epochs!' )
                break
        else:
            earlystop=0
            best_loss=cur_loss
        
    mlflow.pytorch.log_model(model, "pytorch-model")
    model_uri = f"runs:/{run.info.run_id}/model-test"
    model_details = mlflow.register_model(model_uri, 'Mnist')
    mlflow.log_param("model save", model_details.source)
    #print(model_details)
        
    print('Finished Training')
    test_acc={}
    classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    # Test
    correct = 0
    total = 0
    
    model.eval()
    with tqdm(testLoader,leave=True) as tepoch:
        for inputs, labels in tepoch:
            inputs, labels= inputs.to(device), labels.to(device)
            output=model(inputs)
            _, predicted = torch.max(output,1)
            total+=labels.size(0)
            correct += (predicted == labels).sum().item()
            c=(predicted==labels).squeeze()
    
            for i in range(labels.size(0)):
                label=labels[i].item()
                if label not in test_acc:
                    test_acc[label]=[0,0]
                else:
                    test_acc[label][0]+=c[i].item()
                    test_acc[label][1]+=1
    
    
    
    
    for i in range(len(classes)):
        print('Accuracy of %10s : %2d %%' % (classes[i], 100 * test_acc[i][0] / test_acc[i][1]))
    print()
    print('Test Dataset : %d %%' % (100 * correct / total))
    mlflow.log_metric("test accuracy", (100 * correct / total))